In [0]:
%tensorflow_version 2.x
!pip install bert-for-tf2
!pip install -U imbalanced-learn
!pip install sentencepiece
!pip install tensorflow-addons

     |████████████████████████████████| 40kB 4.4MB/s 
  Created wheel for bert-for-tf2: filename=bert_for_tf2-0.14.1-cp36-none-any.whl size=30083 sha256=3747f6af1322a92d10d18a62fd111be7ce5928d743b9b8214b4b91b3031083d9
  Stored in directory: /root/.cache/pip/wheels/dd/f1/10/861fd7899727e4034293fb1dfef45b00f8cd476d21d3b3821e
  Created wheel for py-params: filename=py_params-0.9.7-cp36-none-any.whl size=7302 sha256=2339b6248202973bd09f3709331811ce6fca234362f8e823c0ae2f72dda2a275
  Stored in directory: /root/.cache/pip/wheels/67/f5/19/b461849a50aefdf4bab47c4756596e82ee2118b8278e5a1980
  Created wheel for params-flow: filename=params_flow-0.8.0-cp36-none-any.whl size=15999 sha256=29422457da8c4e435922bdd15ec24bd61960db90ab358afb0dee340343282491
  Stored in directory: /root/.cache/pip/wheels/88/41/05/1a9955d1d01575bbd58aab76e22f8c7eeabba905d551576f43
Successfully built bert-for-tf2 py-params params-flow
     |████████████████████████████████| 163kB 7.7MB/s 
  Found existing installation: imba

In [0]:
import tensorflow as tf
import tensorflow_hub as hub
import bert
from tensorflow.keras.models import  Model
from tqdm import tqdm
import numpy as np
from collections import namedtuple
from sklearn.model_selection import train_test_split
from collections import Counter
from imblearn.over_sampling import RandomOverSampler
from google.colab import files
import os

uploaded = files.upload()
for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] ="/content/gcs.json"

Using TensorFlow backend.


Saving gcs.json to gcs.json
User uploaded file "gcs.json" with length 2297 bytes


In [0]:
!gcloud auth activate-service-account --key-file=/content/gcs.json
!gsutil cp  gs://my-data-bucket-colab/toxic-comments-classification/train.csv.zip train.csv.zip
#!gsutil cp gs://my-data-bucket-colab/toxic-comments-classification/test_labels.csv.zip test_labels.csv.zip 
#!gsutil cp gs://my-data-bucket-colab/toxic-comments-classification/test.csv.zip test.csv.zip 
!unzip train.csv.zip
#!unzip test.csv.zip

Activated service account credentials for: [gcs-40@pivotal001.iam.gserviceaccount.com]
Copying gs://my-data-bucket-colab/toxic-comments-classification/train.csv.zip...
\ [1 files][ 26.3 MiB/ 26.3 MiB]                                                
Operation completed over 1 objects/26.3 MiB.                                     
Archive:  train.csv.zip
  inflating: train.csv               


In [0]:
def RS(k,features_train,ratio=1.0):
    train_y = features_train[:,3*128:390]
    label = train_y[:,k]
    rus = RandomOverSampler(ratio,random_state=42)
    features_train_res, _ = rus.fit_resample(features_train, label)
    np.random.shuffle(features_train_res)
    features_train_res = features_train_res[:100000,:]
    return features_train_res

def results(pred_results,ground_truth,threshold):
    pred_results = (pred_results>=threshold).astype(int)
    print("Positive labels RECALL-thre.{}:{}%".format(threshold,100*np.sum(pred_results*ground_truth)/np.sum(ground_truth)))
    cf = tfa.metrics.MultiLabelConfusionMatrix(num_classes=6)
    cf.update_state(ground_truth, pred_results)
    print('Confusion matrix:', cf.result().numpy())
    hl = tfa.metrics.HammingLoss(mode='multilabel', threshold=threshold)
    hl.update_state(ground_truth, pred_results)
    print('Hamming loss:', hl.result().numpy())


def label_to_class(v):
    w = np.array([32,16,8,4,2,1],dtype=np.int16)
    return np.sum(v*w,axis=1)

def class_to_label(c):
    c=np.reshape(c,(-1,1))
    v5=c%2
    c=c//2
    v4=c%2
    c=c//2
    v3=c%2
    c=c//2
    v2=c%2
    c=c//2
    v1=c%2
    c=c//2
    v0=c%2
    return np.concatenate([v0,v1,v2,v3,v4,v5],axis=1)
def distribution(pattern):
    b=class_to_label(np.array(list(pattern.keys())))
    a=np.asarray(list(pattern.values())).reshape(-1,1)
    M = a*b
    w_vec=np.sum(M,axis=0)
    #return w_vec/np.sum(w_vec)
    return w_vec
def class_count(s,pattern):
    S=0
    for k in list(s):
        S+=pattern[k]
    return S
def _floats_feature(value):
    return tf.train.Feature(float_list=tf.train.FloatList(value=value))

def _int64_feature(value):
    return tf.train.Feature(int64_list=tf.train.Int64List(value=value))

def parse_func(s_example):
    features = {
                'labels':tf.io.FixedLenFeature((6,),tf.float32),
                'masks':tf.io.FixedLenFeature((128,),tf.int64),
		            'ids':tf.io.FixedLenFeature((128,),tf.int64),
		            'segments':tf.io.FixedLenFeature((128,),tf.int64),
                }
    example = tf.io.parse_single_example(s_example, features=features)
    return (tf.cast(example['ids'],tf.int32),tf.cast(example['masks'],tf.int32),tf.cast(example['segments'],tf.int32)),example['labels']

def imgSerialization (ids,masks,segments,labels):
    example = tf.train.Example(features=tf.train.Features(feature={
        'labels':_floats_feature(labels),
        'ids':_int64_feature(ids),
	'masks':_int64_feature(masks),
	'segments':_int64_feature(segments)
        }))
    return example.SerializeToString()


In [0]:
# multi-label hamming loss
import tensorflow_addons as tfa


In [0]:
bert_layer=hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/1",trainable=False)
MAX_SEQ_LEN=128


In [0]:
FullTokenizer=bert.bert_tokenization.FullTokenizer
vocab_file=bert_layer.resolved_object.vocab_file.asset_path.numpy()
do_lower_case=bert_layer.resolved_object.do_lower_case.numpy()
tokenizer=FullTokenizer(vocab_file,do_lower_case)

In [0]:
def get_masks(tokens, max_seq_length):
    return [1]*len(tokens) + [0] * (max_seq_length - len(tokens))

def get_segments(tokens, max_seq_length):
    """Segments: 0 for the first sequence, 1 for the second"""
    segments = []
    current_segment_id = 0
    for token in tokens:
        segments.append(current_segment_id)
        if token == "[SEP]":
            current_segment_id = 1
    return segments + [0] * (max_seq_length - len(tokens))
def get_ids(tokens, tokenizer, max_seq_length):
    """Token ids from Tokenizer vocab"""
    token_ids = tokenizer.convert_tokens_to_ids(tokens,)
    input_ids = token_ids + [0] * (max_seq_length-len(token_ids))
    return input_ids
def create_single_input(sentence,MAX_LEN):
  stokens = tokenizer.tokenize(sentence)
  stokens = stokens[:MAX_LEN]
  stokens = ["[CLS]"] + stokens + ["[SEP]"]
  ids = get_ids(stokens, tokenizer, MAX_SEQ_LEN)
  masks = get_masks(stokens, MAX_SEQ_LEN)
  segments = get_segments(stokens, MAX_SEQ_LEN)
  return ids,masks,segments
def create_input_array(sentences):
  input_ids, input_masks, input_segments = [], [], []
  for sentence in tqdm(sentences,position=0, leave=True):
    ids,masks,segments=create_single_input(sentence,MAX_SEQ_LEN-2)
    input_ids.append(ids)
    input_masks.append(masks)
    input_segments.append(segments)
  return [np.asarray(input_ids, dtype=np.int32), 
            np.asarray(input_masks, dtype=np.int32), 
            np.asarray(input_segments, dtype=np.int32)]

In [0]:
import pandas as pd
df=pd.read_csv('train.csv')
df = df.sample(frac=1)
train_sentences = df["comment_text"].fillna("CVxTz").values
list_classes = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]
inputs=create_input_array(train_sentences)

100%|██████████| 159571/159571 [03:06<00:00, 855.33it/s]


In [0]:
train_y = df[list_classes].values
features=np.concatenate(inputs,axis=1)
features=np.concatenate((features,train_y),axis=1)

features_train,features_test,train_y,_= train_test_split(features, train_y,test_size=0.2,random_state=42)
print("Train set samples:",train_y.shape[0])
print(distribution(Counter(label_to_class(train_y)))/train_y.shape[0])



Train set samples: 127656
[0.09599235 0.00998778 0.05304099 0.00300808 0.04937488 0.00886758]


In [0]:
prior = distribution(Counter(label_to_class(train_y)))/train_y.shape[0]
list_trains = []
features_train_res= RS(3,features_train)
list_trains.append(features_train_res)
features_train_res= RS(5,features_train)
list_trains.append(features_train_res)
features_train_res= RS(1,features_train)
list_trains.append(features_train_res)
features_train_res= RS(4,features_train)
list_trains.append(features_train_res)
features_train_res= RS(2,features_train)
list_trains.append(features_train_res)
features_train_res= RS(0,features_train)
list_trains.append(features_train_res)

features_train=np.concatenate(list_trains,axis=0)
np.random.shuffle(features_train)
train_y = features_train[:,384:390]
print(train_y.shape)
after = distribution(Counter(label_to_class(train_y)))/600000
print(after/prior)
print(after)
np.max(train_y,axis=1).sum()

(600000, 6)
[ 5.32097932 16.25936951  7.52493565 34.86936937  7.92889651 16.71312498]
[0.51077333 0.162395   0.39913    0.10489    0.39148833 0.148205  ]


320508

In [0]:
X = np.split(features_train,[128,2*128,3*128,390],axis=1)
input_ids=X[0]
input_masks=X[1]
input_segments=X[2]
train_y= X[3].astype(np.float32)

writer = tf.io.TFRecordWriter('gs://my-data-bucket-colab/toxic-comments-classification/train.tfrecords')
for i in range(len(train_y)):
      s_example = imgSerialization(input_ids[i,:],input_masks[i,:],input_segments[i,:],train_y[i,:])
      writer.write(s_example)
writer.close()

X = np.split(features_test,[128,2*128,3*128,390],axis=1)
input_ids_test=X[0]
input_masks_test=X[1]
input_segments_test=X[2]
test_y= X[3].astype(np.float32)

writer = tf.io.TFRecordWriter('gs://my-data-bucket-colab/toxic-comments-classification/test.tfrecords')
for i in range(len(test_y)):
      s_example = imgSerialization(input_ids_test[i,:],input_masks_test[i,:],input_segments_test[i,:],test_y[i,:])
      writer.write(s_example)
writer.close()

